import libraries

In [5]:
import pandas as pd
import requests
import config


In [9]:

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EHAM/2023-03-28T20:00/2023-03-29T08:00"

querystring = {"direction":"Arrival","withCancelled":"true","withCodeshared":"false","withLocation":"false"}

headers = {
	"X-RapidAPI-Key": config.flights_API_key,
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"arrivals":[{"movement":{"airport":{"icao":"UBBB","iata":"GYD","name":"Baku"},"scheduledTimeLocal":"2023-03-28 10:30+02:00","actualTimeLocal":"2023-03-28 20:02+02:00","runwayTimeLocal":"2023-03-28 20:02+02:00","scheduledTimeUtc":"2023-03-28 08:30Z","actualTimeUtc":"2023-03-28 18:02Z","runwayTimeUtc":"2023-03-28 18:02Z","quality":["Basic","Live"]},"number":"7L 211","status":"Arrived","codeshareStatus":"IsOperator","isCargo":true,"aircraft":{"model":"Boeing 747-400 Freighter"},"airline":{"name":"Silk Way West","iata":"7L","icao":"AZG"}},{"movement":{"airport":{"icao":"ZSOF","iata":"HFE","name":"Hefei"},"scheduledTimeLocal":"2023-03-28 18:30+02:00","actualTimeLocal":"2023-03-28 20:33+02:00","runwayTimeLocal":"2023-03-28 20:33+02:00","scheduledTimeUtc":"2023-03-28 16:30Z","actualTimeUtc":"2023-03-28 18:33Z","runwayTimeUtc":"2023-03-28 18:33Z","runway":"18R","quality":["Basic","Live"]},"number":"Y8 7417","callSign":"YZR7417","status":"Arrived","codeshareStatus":"IsOperator","isCargo":true,

In [10]:
response_arrivals = response.json()



In [11]:
response_arrivals['arrivals'][0].keys()

dict_keys(['movement', 'number', 'status', 'codeshareStatus', 'isCargo', 'aircraft', 'airline'])

In [12]:
#accessing local time arrival
response_arrivals['arrivals'][0]['movement']['scheduledTimeLocal']

'2023-03-28 10:30+02:00'

In [53]:
response_arrivals['arrivals'][0]

{'movement': {'airport': {'icao': 'UBBB', 'iata': 'GYD', 'name': 'Baku'},
  'scheduledTimeLocal': '2023-03-28 10:30+02:00',
  'actualTimeLocal': '2023-03-28 20:02+02:00',
  'runwayTimeLocal': '2023-03-28 20:02+02:00',
  'scheduledTimeUtc': '2023-03-28 08:30Z',
  'actualTimeUtc': '2023-03-28 18:02Z',
  'runwayTimeUtc': '2023-03-28 18:02Z',
  'quality': ['Basic', 'Live']},
 'number': '7L 211',
 'status': 'Arrived',
 'codeshareStatus': 'IsOperator',
 'isCargo': True,
 'aircraft': {'model': 'Boeing 747-400 Freighter'},
 'airline': {'name': 'Silk Way West', 'iata': '7L', 'icao': 'AZG'}}

In [21]:
#accessing city name
response_arrivals['arrivals'][0]['movement']['airport']['name']

'Baku'

In [25]:
# accessing ICAO code
response_arrivals['arrivals'][0]['movement']['airport']['icao']

'UBBB'

In [82]:
response_arrivals['arrivals'][0]['movement']['actualTimeLocal']

'2023-03-28 20:02+02:00'

In [8]:
print(response.json)

<bound method Response.json of <Response [200]>>


In [13]:
from IPython.display import JSON

In [ ]:
display(JSON(response.json()))

dataframes for cities and airports

In [15]:
arrivals  = pd.json_normalize(response.json()['arrivals'])

In [23]:
arrivals[arrivals['movement.airport.icao'].isin(['EDDF', 'LTFM'])][['movement.airport.icao', 'movement.airport.name']]\
    .drop_duplicates().rename(columns={'movement.airport.icao': 'ICAO', 'movement.airport.name': 'Airport Name'}).reset_index(drop=True)

,ICAO,Airport Name
0,EDDF,Frankfurt-am-Main
1,LTFM,Istanbul


In [102]:
arrivals[arrivals['movement.airport.icao'].isin(['EDDF', 'EDDS', 'EDDB'])]

,number,status,codeshareStatus,isCargo,movement.airport.icao,movement.airport.iata,movement.airport.name,movement.scheduledTimeLocal,movement.actualTimeLocal,movement.runwayTimeLocal,...,aircraft.model,airline.name,airline.iata,airline.icao,callSign,movement.runway,aircraft.reg,aircraft.modeS,movement.terminal,movement.baggageBelt
16,KL 1774,Arrived,IsOperator,False,EDDF,FRA,Frankfurt-am-Main,2023-03-28 20:25+02:00,2023-03-28 20:20+02:00,2023-03-28 20:20+02:00,...,Embraer 175,KLM,KL,KLM,KLM1774,NaN,PH-EXS,485779,2,8
64,LH 1002,Arrived,IsOperator,False,EDDF,FRA,Frankfurt-am-Main,2023-03-28 22:15+02:00,2023-03-28 22:23+02:00,2023-03-28 22:23+02:00,...,Airbus A320,Lufthansa,LH,DLH,DLH5AK,18R,D-AIZD,3C6744,1,2
70,KL 1834,Arrived,IsOperator,False,EDDB,BER,Berlin,2023-03-28 22:05+02:00,2023-03-28 21:50+02:00,2023-03-28 21:50+02:00,...,Embraer 190,KLM,KL,KLM,KLM70D,18R,PH-EZZ,485065,2,12
134,KL 1818,Arrived,IsOperator,False,EDDB,BER,Berlin,2023-03-29 07:20+02:00,2023-03-29 07:11+02:00,2023-03-29 07:11+02:00,...,Boeing 737-800 (winglets),KLM,KL,KLM,KLM44R,18R,PH-BXU,484442,2,11
136,KL 1866,Arrived,IsOperator,False,EDDS,STR,Stuttgart,2023-03-29 07:25+02:00,2023-03-29 07:30+02:00,2023-03-29 07:30+02:00,...,Embraer 190,KLM,KL,KLM,KLM68W,NaN,D-AZFA,3C1AEB,2,8
161,KL 1762,Arrived,IsOperator,False,EDDF,FRA,Frankfurt-am-Main,2023-03-29 08:10+02:00,2023-03-29 07:57+02:00,2023-03-29 07:57+02:00,...,Embraer 190,KLM,KL,KLM,KLM1762,NaN,PH-EZW,484F7F,2,12


In [28]:
# create a new df cities with the cities based on arrivals df filtering movement.airport.icao and airport.name columns and renaming them into new columns icao and city

arrivals[arrivals['movement.airport.icao'].isin(['EDDF', 'LTFM', 'EDDB'])][['movement.airport.icao', 'movement.airport.name']].drop_duplicates()

cities_df = arrivals[['movement.airport.icao', 'movement.airport.name']].drop_duplicates()

# rename columns as icao and city



cities_df = pd.DataFrame({'city': ['Frankfurt', 'Munich', 'Berlin'], 'icao': ['EDDF', 'EDDM', 'EDDB']})
cities_df

,city,icao
0,Frankfurt,EDDF
1,Munich,EDDM
2,Berlin,EDDB


In [13]:
arrivals.sample(5)

,number,callSign,status,codeshareStatus,isCargo,movement.airport.icao,movement.airport.iata,movement.airport.name,movement.scheduledTimeLocal,movement.actualTimeLocal,...,movement.runway,movement.quality,aircraft.reg,aircraft.modeS,aircraft.model,airline.name,airline.iata,airline.icao,movement.terminal,movement.baggageBelt
31,FR 2334,NaN,Arrived,IsOperator,False,LEMG,AGP,Málaga,2023-03-28 21:25+02:00,2023-03-28 21:34+02:00,...,NaN,"[Basic, Live]",NaN,NaN,Boeing 737-800 (winglets),Ryanair,FR,RYR,4,22
145,KL 1866,KLM68W,Arrived,IsOperator,False,EDDS,STR,Stuttgart,2023-03-29 07:25+02:00,2023-03-29 07:30+02:00,...,NaN,"[Basic, Live]",D-AZFA,3C1AEB,Embraer 190,KLM,KL,KLM,2,8
36,KL 1150,KLM54B,Arrived,IsOperator,False,ENGM,OSL,Oslo,2023-03-28 21:05+02:00,2023-03-28 21:03+02:00,...,NaN,"[Basic, Live]",PH-EZB,484ACC,Embraer 190,KLM,KL,KLM,2,12
44,KL 994,KLM30B,Arrived,IsOperator,False,EGLC,LCY,London,2023-03-28 21:20+02:00,2023-03-28 20:56+02:00,...,NaN,"[Basic, Live]",PH-EZN,484C52,Embraer 190,KLM,KL,KLM,2,13
111,KL 652,KLM292,Arrived,IsOperator,False,KIAD,IAD,District of Columbia,2023-03-29 06:45+02:00,2023-03-29 06:21+02:00,...,18R,"[Basic, Live]",PH-AKA,484F71,Airbus A330-300,KLM,KL,KLM,2,12


arrivals dataFrame with one city

In [74]:
airport = 'EDDS'
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport}/2023-03-28T20:00/2023-03-29T08:00"

querystring = {"direction":"Arrival","withCancelled":"true","withCodeshared":"false","withLocation":"false"}

headers = {
	"X-RapidAPI-Key": config.flights_API_key,
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

In [31]:
city_response = response.json()
city_response

{'arrivals': [{'movement': {'airport': {'icao': 'UBBB',
     'iata': 'GYD',
     'name': 'Baku'},
    'scheduledTimeLocal': '2023-03-28 10:30+02:00',
    'actualTimeLocal': '2023-03-28 20:02+02:00',
    'runwayTimeLocal': '2023-03-28 20:02+02:00',
    'scheduledTimeUtc': '2023-03-28 08:30Z',
    'actualTimeUtc': '2023-03-28 18:02Z',
    'runwayTimeUtc': '2023-03-28 18:02Z',
    'quality': ['Basic', 'Live']},
   'number': '7L 211',
   'status': 'Arrived',
   'codeshareStatus': 'IsOperator',
   'isCargo': True,
   'aircraft': {'model': 'Boeing 747-400 Freighter'},
   'airline': {'name': 'Silk Way West', 'iata': '7L', 'icao': 'AZG'}},
  {'movement': {'airport': {'icao': 'ZSOF', 'iata': 'HFE', 'name': 'Hefei'},
    'scheduledTimeLocal': '2023-03-28 18:30+02:00',
    'actualTimeLocal': '2023-03-28 20:33+02:00',
    'runwayTimeLocal': '2023-03-28 20:33+02:00',
    'scheduledTimeUtc': '2023-03-28 16:30Z',
    'actualTimeUtc': '2023-03-28 18:33Z',
    'runwayTimeUtc': '2023-03-28 18:33Z',
    

In [32]:
city_response['arrivals'][0]



{'movement': {'airport': {'icao': 'UBBB', 'iata': 'GYD', 'name': 'Baku'},
  'scheduledTimeLocal': '2023-03-28 10:30+02:00',
  'actualTimeLocal': '2023-03-28 20:02+02:00',
  'runwayTimeLocal': '2023-03-28 20:02+02:00',
  'scheduledTimeUtc': '2023-03-28 08:30Z',
  'actualTimeUtc': '2023-03-28 18:02Z',
  'runwayTimeUtc': '2023-03-28 18:02Z',
  'quality': ['Basic', 'Live']},
 'number': '7L 211',
 'status': 'Arrived',
 'codeshareStatus': 'IsOperator',
 'isCargo': True,
 'aircraft': {'model': 'Boeing 747-400 Freighter'},
 'airline': {'name': 'Silk Way West', 'iata': '7L', 'icao': 'AZG'}}

arrivals dataFrame with multiple cities

In [ ]:
#an example how to filter data with if statement
# arrivals = []
# for flight in json_data[0]:
#     if flight['movement']['airport']['iata'] == 'STR': #and flight['status'] == 'Arrived':
#         arrivals.append({
#             'Departure Airport': flight['movement']['airport']['name'],
#             'Actual Arrival': flight['movement']['actualTimeLocal'],
#             'Scheduled Arrival': flight['movement']['scheduledTimeLocal']
#         })

# # Create DataFrame
# arrivals_df_two = pd.DataFrame(arrivals)

# Print DataFrame


In [150]:
# icao codes for airports frankfurt, istanbul, stuttgart
airports = ['EDDF', 'LTFM']

# create a list of dictionaries
arrivals_dict = {
    'icao': [],
    'city': [],
    'actual_arrival': [],
    'scheduled_arrival': [],
    'terminal': [],
    'airline': []
}

# loop through the airports list
for airport in airports:
    # the url, querystring and headers were created in rapidapi website and copied here
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport}/2023-03-28T20:00/2023-03-29T08:00"
    querystring = {"direction":"Arrival","withCancelled":"true","withCodeshared":"false","withLocation":"false"}
    
    headers = {
        "X-RapidAPI-Key": config.flights_API_key,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }
    # make the request
    response = requests.request("GET", url, headers=headers, params=querystring)
    # get the json data
    
    json_data = response.json()['arrivals']
    
    for flight in json_data:
        arrivals_dict['icao'].append(flight['movement']['airport'].get("icao", None))
        arrivals_dict['city'].append(flight['movement']['airport'].get('name', None))
        arrivals_dict['actual_arrival'].append(flight['movement'].get('name', None))
        arrivals_dict['scheduled_arrival'].append(flight['movement'].get('scheduledTimeLocal', None))
        arrivals_dict['terminal'].append(flight['movement'].get('terminal', None))
        arrivals_dict['airline'].append(flight['airline'].get('name', None))    
    

    
    # json_data = response.json()

    # loop through the json data
    # for flight in json_data['arrivals']:
        # arrivals_dict['icao'].append(flight['movement']['airport']['icao'])
        # arrivals_dict['city'].append(flight['movement']['airport']['name'])
        # arrivals_dict['actual_arrival'].append(flight['movement']['actualTimeLocal'])
        # arrivals_dict['scheduled_arrival'].append(flight['movement']['scheduledTimeLocal'])
        # arrivals_dict['terminal'].append(flight['movement']['terminal'])
        # arrivals_dict['airline'].append(flight['airline']['name'])

        # arrivals_flights_dict = {}
        # arrivals_flights_dict['arrival_icao'] = airport
        # arrivals_flights_dict['scheduled_flight'] = flight[0]['movement']['scheduledTimeLocal']

    # # alternatively, you can use (i in range(len(json_data['arrivals']))):
    # for i in range(len(json_data['arrivals'])):
    #     arrivals_dict['icao'] = i['movement']['airport'].get('icao', None)
        # arrivals_dict['city'].append(json_data['arrivals'][i]['movement']['airport']['name'])
        # arrivals_dict['actual_arrival'].append(json_data['arrivals'][i]['movement']['actualTimeLocal'])
        # arrivals_dict['scheduled_arrival'].append(json_data['arrivals'][i]['movement']['scheduledTimeLocal'])
        # arrivals_dict['terminal'].append(json_data['arrivals'][i]['movement']['terminal'])
        # arrivals_dict['airline'].append(json_data['arrivals'][i]['airline']['name'])

# create the DataFrame
arrivals_df = pd.DataFrame(arrivals_dict)

In [36]:
#THIS VERSION HAS ARRIVAL AIRPORTS
#indentation is very important, otherwiser some elements not included in the dictionary and ger overwritten by the next iteration

# icao codes for airports frankfurt, istanbul, stuttgart
airports = ['EDDF', 'LTFM']

arrivals_dict = {
    'icao': [],
    'city': [],
    'actual_arrival': [],
    'scheduled_arrival': [],
    'terminal': [],
    'airline': [],
    'airport': []
    }

# loop through the airports list
for airport in airports:
    # the url, querystring and headers were created in rapidapi website and copied here
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport}/2023-03-28T20:00/2023-03-29T08:00"
    querystring = {"direction":"Arrival","withCancelled":"true","withCodeshared":"false","withLocation":"false"}
    
    headers = {
        "X-RapidAPI-Key": config.flights_API_key,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }
    # make the request
    response = requests.request("GET", url, headers=headers, params=querystring)
    # get the json data
    json_data = response.json()['arrivals']
    
    for flight in json_data:
        # if flight['movement']['airport'].get("icao") in airports:
            arrivals_dict['icao'].append(flight['movement']['airport'].get("icao", None))
            arrivals_dict['city'].append(flight['movement']['airport'].get('name', None))
            arrivals_dict['actual_arrival'].append(flight['movement'].get('scheduledTimeUtc', None))
            arrivals_dict['scheduled_arrival'].append(flight['movement'].get('scheduledTimeLocal', None))
            arrivals_dict['terminal'].append(flight['movement'].get('terminal', None))
            arrivals_dict['airline'].append(flight['airline'].get('name', None))
            arrivals_dict['airport'].append(airport)  

arrivals_df = pd.DataFrame(arrivals_dict)

In [215]:
# here indentation prevents all airports from being added to the dataframe. The last airport LTFM overwrites the former airports in the list

for flight in json_data:
        # if flight['movement']['airport'].get("icao") in airports:
    arrivals_dict['icao'].append(flight['movement']['airport'].get("icao", None))
    arrivals_dict['city'].append(flight['movement']['airport'].get('name', None))
    arrivals_dict['actual_arrival'].append(flight['movement'].get('scheduledTimeUtc', None))
    arrivals_dict['scheduled_arrival'].append(flight['movement'].get('scheduledTimeLocal', None))
    arrivals_dict['terminal'].append(flight['movement'].get('terminal', None))
    arrivals_dict['airline'].append(flight['airline'].get('name', None))
    arrivals_dict['airport'].append(airport)  

arrivals_df = pd.DataFrame(arrivals_dict)

In [37]:
arrivals_df.sample(30)


,icao,city,actual_arrival,scheduled_arrival,terminal,airline,airport
94,EDDV,Hanover,2023-03-28 22:46Z,2023-03-29 00:46+02:00,None,Unknown/Private owner,EDDF
222,LUKK,Chişinău,2023-03-28 19:45Z,2023-03-28 22:45+03:00,None,Turkish,LTFM
140,OOMS,Muscat,2023-03-29 05:05Z,2023-03-29 07:05+02:00,2,Oman Air,EDDF
110,KCLT,Charlotte,2023-03-29 05:20Z,2023-03-29 07:20+02:00,2,American,EDDF
66,LTFM,Istanbul,2023-03-28 19:00Z,2023-03-28 21:00+02:00,1,Turkish,EDDF
354,RJTT,Tokyo,2023-03-29 02:15Z,2023-03-29 05:15+03:00,None,Turkish,LTFM
295,LTBS,Dalaman,2023-03-28 21:05Z,2023-03-29 00:05+03:00,None,Turkish,LTFM
155,None,Medinah,2023-03-28 17:45Z,2023-03-28 20:45+03:00,None,Pakistan International,LTFM
301,None,Adana,2023-03-28 21:45Z,2023-03-29 00:45+03:00,None,Turkish,LTFM
158,LTAR,Sivas,2023-03-28 17:50Z,2023-03-28 20:50+03:00,None,Turkish,LTFM


In [33]:
arrivals_dict = {
    'icao': [],
    'city': [],
    'actual_arrival': [],
    'scheduled_arrival': [],
    'terminal': [],
    'airline': [],
    'airport': []
}

for flight in json_data:
    # if flight['movement']['airport'].get("icao") in airports:
        arrivals_dict['icao'].append(flight['movement']['airport'].get("icao", None))
        arrivals_dict['city'].append(flight['movement']['airport'].get('name', None))
        arrivals_dict['actual_arrival'].append(flight['movement'].get('name', None))
        arrivals_dict['scheduled_arrival'].append(flight['movement'].get('scheduledTimeLocal', None))
        arrivals_dict['terminal'].append(flight['movement'].get('terminal', None))
        arrivals_dict['airline'].append(flight['airline'].get('name', None))
        arrivals_dict['airport'].append(airport)  

arrivals_port_df = pd.DataFrame(arrivals_dict)

NameError: name 'json_data' is not defined

In [214]:
arrivals_port_df

,icao,city,actual_arrival,scheduled_arrival,terminal,airline,airport
0,EKCH,Copenhagen,None,2023-03-28 19:15+03:00,None,Turkish,LTFM
1,None,Minsk,None,2023-03-28 20:35+03:00,None,Belavia,LTFM
2,URSS,Sochi,None,2023-03-28 20:30+03:00,None,Azimuth,LTFM
3,LFPG,Paris,None,2023-03-28 20:45+03:00,None,Turkish,LTFM
4,UTTT,Tashkent,None,2023-03-28 20:10+03:00,None,Uzbekistan Airways,LTFM
...,...,...,...,...,...,...,...
254,LTBJ,İzmir,None,2023-03-29 07:40+03:00,None,Turkish,LTFM
255,OITT,Tabriz,None,2023-03-29 07:15+03:00,None,Turkish,LTFM
256,None,Samara,None,2023-03-29 07:25+03:00,None,Red Wings,LTFM
257,URSS,Sochi,None,2023-03-29 08:00+03:00,None,Azimuth,LTFM


In [139]:
json_data

[{'movement': {'airport': {'icao': 'BIKF', 'iata': 'KEF', 'name': 'Reykjavik'},
   'scheduledTimeLocal': '2023-03-28 05:55+02:00',
   'actualTimeLocal': '2023-03-28 20:05+02:00',
   'scheduledTimeUtc': '2023-03-28 03:55Z',
   'actualTimeUtc': '2023-03-28 18:05Z',
   'terminal': '1',
   'gate': 'A1',
   'quality': ['Basic', 'Live']},
  'number': 'LH 869',
  'callSign': 'DLH869',
  'status': 'Arrived',
  'codeshareStatus': 'IsOperator',
  'isCargo': False,
  'aircraft': {'reg': 'D-AILI', 'modeS': '3C6589', 'model': 'Airbus A319'},
  'airline': {'name': 'Lufthansa', 'iata': 'LH', 'icao': 'DLH'}},
 {'movement': {'airport': {'icao': 'LOWL', 'iata': 'LNZ', 'name': 'Linz'},
   'scheduledTimeLocal': '2023-03-28 15:35+02:00',
   'actualTimeLocal': '2023-03-28 20:10+02:00',
   'scheduledTimeUtc': '2023-03-28 13:35Z',
   'actualTimeUtc': '2023-03-28 18:10Z',
   'terminal': '1',
   'gate': 'B1',
   'quality': ['Basic', 'Live']},
  'number': 'EN 8043',
  'callSign': 'DLA8JT',
  'status': 'Arrived',

In [38]:
json_data['arrivals'][0]['movement']['scheduledTimeLocal']

TypeError: list indices must be integers or slices, not str

In [39]:
json_data['arrivals'][0]['movement']['terminal']

TypeError: list indices must be integers or slices, not str

In [112]:
json_data['arrivals'][0]

{'movement': {'airport': {'icao': 'BIKF', 'iata': 'KEF', 'name': 'Reykjavik'},
  'scheduledTimeLocal': '2023-03-28 05:55+02:00',
  'actualTimeLocal': '2023-03-28 20:05+02:00',
  'scheduledTimeUtc': '2023-03-28 03:55Z',
  'actualTimeUtc': '2023-03-28 18:05Z',
  'terminal': '1',
  'gate': 'A1',
  'quality': ['Basic', 'Live']},
 'number': 'LH 869',
 'callSign': 'DLH869',
 'status': 'Arrived',
 'codeshareStatus': 'IsOperator',
 'isCargo': False,
 'aircraft': {'reg': 'D-AILI', 'modeS': '3C6589', 'model': 'Airbus A319'},
 'airline': {'name': 'Lufthansa', 'iata': 'LH', 'icao': 'DLH'}}

In [113]:
json_data['arrivals'][0]['movement']['scheduledTimeLocal']

'2023-03-28 05:55+02:00'

In [ ]:
['arrivals'][0]['movement']['airport']['icao'] #icao code
['arrivals'][0]['movement']['airport']['name'] #city name
['arrivals'][0]['movement']['scheduledTimeLocal'] #scheduled local time arrival
['arrivals'][0]['movement']['actualTimeLocal'] #actual local time arrival
['arrivals'][0]['movement']['terminal']#terminal
['arrivals'][0]['airline'] #airline


version with attempt to to iterate airports in a list

In [134]:

airports = ['EDDF', 'EDDB']

list_df_arrivals = []

for i in range(len(airports)):
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airports[i]}/2023-03-28T20:00/2023-03-29T08:00"

    querystring = {"direction":"Arrival","withCancelled":"true","withCodeshared":"false","withLocation":"false"}

    headers = {
        "X-RapidAPI-Key": config.flights_API_key,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    list_df_arrivals.append(response.json()['arrivals'])
    
    # list_df_arrivals.append(pd.json_normalize(response.json()['arrivals']))
    
    # list_df_arrivals.append(arrivals[arrivals['movement.airport.icao'].isin([airports[i]])])





In [129]:
list_df_arrivals.append(response.json()['arrivals'])

In [141]:
list_df_arrivals

[[{'movement': {'airport': {'icao': 'BIKF',
     'iata': 'KEF',
     'name': 'Reykjavik'},
    'scheduledTimeLocal': '2023-03-28 19:25+02:00',
    'actualTimeLocal': '2023-03-28 20:00+02:00',
    'scheduledTimeUtc': '2023-03-28 17:25Z',
    'actualTimeUtc': '2023-03-28 18:00Z',
    'terminal': '1',
    'gate': 'A1',
    'quality': ['Basic', 'Live']},
   'number': 'LH 857',
   'callSign': 'DLH2CK',
   'status': 'Arrived',
   'codeshareStatus': 'IsOperator',
   'isCargo': False,
   'aircraft': {'reg': 'D-AIZQ',
    'modeS': '3C6751',
    'model': 'Airbus A320 (Sharklets)'},
   'airline': {'name': 'Lufthansa', 'iata': 'LH', 'icao': 'DLH'}},
  {'movement': {'airport': {'icao': 'LSGG', 'iata': 'GVA', 'name': 'Geneva'},
    'scheduledTimeLocal': '2023-03-28 19:55+02:00',
    'actualTimeLocal': '2023-03-28 20:02+02:00',
    'scheduledTimeUtc': '2023-03-28 17:55Z',
    'actualTimeUtc': '2023-03-28 18:02Z',
    'terminal': '1',
    'gate': 'A2',
    'quality': ['Basic', 'Live']},
   'number': '

In [205]:
list_df_arrivals[1][0]['movement']['airport']['name']

'Faro'

In [107]:
list_df_arrivals[0]['movement']['airport']

{'icao': 'BIKF', 'iata': 'KEF', 'name': 'Reykjavik'}

In [108]:
list_df_arrivals[0]['movement']

{'airport': {'icao': 'BIKF', 'iata': 'KEF', 'name': 'Reykjavik'},
 'scheduledTimeLocal': '2023-03-28 19:25+02:00',
 'actualTimeLocal': '2023-03-28 20:00+02:00',
 'scheduledTimeUtc': '2023-03-28 17:25Z',
 'actualTimeUtc': '2023-03-28 18:00Z',
 'terminal': '1',
 'gate': 'A1',
 'quality': ['Basic', 'Live']}

In [167]:
arrivals_dict = {}
arrivals_dict['departure_airport'] = list_df_arrivals[0]['movement']['airport']['name']
arrivals_dict['scheduled_arrival'] = list_df_arrivals[0]['movement']['scheduledTimeLocal']
arrivals_dict['actual_arrival'] = list_df_arrivals[0]['movement']['actualTimeLocal']
arrivals_dict['airline_name'] = list_df_arrivals[0]['airline']['name']
arrivals_dict['terminal'] = list_df_arrivals[0]['movement']['terminal']

arrivals_dict


TypeError: list indices must be integers or slices, not str

In [182]:
list_for_df = []

for i in range(len(list_df_arrivals)):
    arrivals_dict = {}
    arrivals_dict['departure_city'] = list_df_arrivals[i]['movement']['airport']['name']
    # arrivals_dict['scheduled_arrival'] = list_df_arrivals[i]['movement']['scheduledTimeLocal']
    # arrivals_dict['actual_arrival'] = list_df_arrivals[i]['movement']['actualTimeLocal']
    # arrivals_dict['airline_name'] = list_df_arrivals[i]['airline']['name']
    # arrivals_dict['terminal'] = list_df_arrivals[i]['movement']['terminal']
    list_for_df.append(arrivals_dict)

TypeError: list indices must be integers or slices, not str

In [5]:
list_for_df = []

for i in list_df_arrivals:
    arrivals_dict = {}
    arrivals_dict['departure_city'] = i[0]['movement']['airport']['name']
    arrivals_dict['scheduled_arrival'] = i[0]['movement']['scheduledTimeLocal']
    # arrivals_dict['actual_arrival'] = i[0]['movement']['actualTimeLocal']
    # arrivals_dict['airline_name'] = i[0]['airline']['name']
    # arrivals_dict['terminal'] = i[0]['movement']['terminal']
    list_for_df.append(arrivals_dict)


NameError: name 'list_df_arrivals' is not defined

In [159]:
i['movement']['airport']['name']

TypeError: list indices must be integers or slices, not str

In [6]:
list_for_df

[]

In [202]:
arrivals_df = pd.DataFrame(list_for_df)


In [203]:
arrivals_df

,departure_city,scheduled_arrival
0,Reykjavik,2023-03-28 19:25+02:00
1,Faro,2023-03-28 20:05+02:00
